In [2]:
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.cross_validation import train_test_split
from statsmodels.discrete.discrete_model import Logit
from pandas.tools.plotting import scatter_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score
import pandas as pd
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

In [7]:
league_pit = pd.read_csv('/home/jun/Galvanize/capstone/Galvanize_capstone/Data/League_stats/league_stats_all_pit.csv')

In [8]:
league_pit = league_pit.drop('Unnamed: 0', axis=1)

In [3]:
X = pd.read_csv('X_pitching.csv')

In [4]:
X.shape

(95620, 32)

In [5]:
X = X.drop('Unnamed: 0', axis=1)

# Going to have a different approach to do feature engineering, just like second_feat_eng.ipynb

In [28]:
X.Drafted = X.Drafted.apply(lambda x: 100 if x==0 else x)  # for undrafted players

In [29]:
X.insert(0, 'Number_row', 1)

In [31]:
X = X[X.Lg_pitching != 'MEX']

In [39]:
X = X[X.Lg_pitching != 'DOSL']

In [42]:
X = X[X.Lg_pitching != 'VESL']

In [46]:
X = X[X.Lg_pitching != 'AA']

In [40]:
league_pit.Lg.unique()

array(['International League', 'Pacific Coast League', 'Eastern League',
       'Southern League', 'Texas League', 'California League',
       'Carolina League', 'Florida State League', 'Midwest League',
       'South Atlantic League', 'New York-Pennsylvania League',
       'Northwest League', 'Appalachian League', 'Arizona League',
       'Gulf Coast League', 'Pioneer League'], dtype=object)

In [47]:
X.Lg_pitching.unique()

array(['ARIZ', 'MIDW', 'FLOR', 'PCL', 'APPY', 'NYPL', 'SALL', 'EL', 'NORW',
       'SOUL', 'IL', 'PION', 'CALL', 'TL', 'CARL', 'GULF'], dtype=object)

In [49]:
Lg_abbr = {'International League': 'IL', 'Pacific Coast League': 'PCL', 'Eastern League': 'EL', 'Arizona League': 'ARIZ',
 'Midwest League': 'MIDW', 'Florida State League': 'FLOR', 'Appalachian League': 'APPY', 'New York-Pennsylvania League': 'NYPL',
 'South Atlantic League': 'SALL', 'Northwest League': 'NORW', 'Southern League': 'SOUL', 'Pioneer League': 'PION',
 'California League': 'CALL', 'Texas League': 'TL', 'Carolina League': 'CARL', 'Gulf Coast League': 'GULF'}
 

In [51]:
league_pit.Lg = league_pit.Lg.apply(lambda x: Lg_abbr[x])

In [ ]:
'AgeDif_pitching', 'Age_pitching' 'BF', 'Drafted', 'G_pitching', 'Height', 'IP', 'Rookie?', 'Throws', 'Weight'# simple avearge (number_row)

In [ ]:
X[['L', 'SV', 'W', 'WP']] = \
    X[['L', 'SV', 'W', 'WP']].apply(lambda x: x * X.G_pitching)# weighted average (game)

In [ ]:
X[['ERA', 'HR9', 'SO/W', 'SO9', 'WHIP']] =\
    X[['ERA', 'HR9', 'SO/W', 'SO9', 'WHIP']].apply(lambda x: x * X.IP) # weighted average (IP)

In [ ]:
X = X.drop(['BB9', 'GS', 'H9', 'RAvg'], axis=1) # drop

In [ ]:
X_group = X.groupby(['player_id', 'Year']).sum()

In [ ]:
X_group.Lev = X_group.Lev / X_group.Number_row + X_group.Number_row

In [ ]:
X_group = X_group[['Number_row', 'Lev', 'AgeDif_pitching', 'Age_pitching', 
'BF', 'Drafted', 'ERA', 'G_pitching',
'HR9', 'Height', 'IP', 'L', 'Rookie?',
'SO/W', 'SO9', 'SV', 'Throws', 'W', 'WHIP', 'WP', 'Weight']]

In [ ]:
X_group[['AgeDif_pitching', 'Age_pitching', 'BF', 'Drafted', 'G_pitching', 'Height', 'IP', 'Rookie?', 'Throws', 'Weight']] =\
X_group[['AgeDif_pitching', 'Age_pitching', 'BF', 'Drafted', 'G_pitching', 'Height', 'IP', 'Rookie?', 'Throws', 'Weight']].apply(lambda x: x / X_group.Number_row)

In [ ]:
X_group[['L', 'SV', 'W', 'WP']] = X_group[['L', 'SV', 'W', 'WP']].apply(lambda x: x / X_group.G_pitching)

In [ ]:
X_group[['ERA', 'HR9', 'SO/W', 'SO9', 'WHIP']] =\
    X_group[['ERA', 'HR9', 'SO/W', 'SO9', 'WHIP']].apply(lambda x: x / X_group.IP)

In [ ]:
X_group_copy = X_group.copy()

In [ ]:
X_group_copy

In [ ]:
y_ver3 = pd.DataFrame([0], columns=['0.0']).append(y_ver2, ignore_index=True)

In [ ]:
type(y_ver2)

In [ ]:
X_group.to_csv('X_ver2.csv')

In [ ]:
y_ver2 = pd.read_csv('y_ver2.csv')

In [ ]:
X_ver2 = pd.read_csv('X_ver2.csv')

In [ ]:
X_ver2 = X_ver2.drop(['player_id', 'Year'], axis=1)

In [ ]:
y_ver2 = y_ver2.drop(['0', '2011'], axis=1)

In [ ]:
y_ver3 = y_ver3['0.0']

In [ ]:
y_ver3.shape

In [ ]:
X_group.shape

In [ ]:
y_ver3.shape

-----

In [ ]:
X.POS = X.POS.apply(lambda x: 0 if x=='c' else x)

In [ ]:
X.POS = X.POS.apply(lambda x: 1 if x=='rp' else x)

In [ ]:
X.POS = X.POS.apply(lambda x: 2 if x=='sp' else x)

In [ ]:
X.insert(X.shape[1], 'BMI', (X.Weight * 0.45) / ((X.Height*0.01)**2))

In [ ]:
X = X.drop(['Weight', 'Height'], axis=1)

In [ ]:
X = X.loc[:, X.dtypes!=object]

In [ ]:
X.BAOpp = X.BAOpp.fillna(X.BAOpp.mean())

In [ ]:
X = X.drop(['Year', 'player_id', 'yearID', 'yearID.1'], axis=1)

In [ ]:
#X = X.drop('Pre_lev', axis=1)

In [ ]:
X.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
corr = np.corrcoef(X.values, rowvar=0)

In [ ]:
corr.shape

In [ ]:
w, v = np.linalg.eig(corr)

In [ ]:
np.argsort(w)

In [ ]:
np.argsort(v[:,0])

In [ ]:
X.columns[7]

In [ ]:
y.unique()

In [ ]:
np.any(np.isnan(X))

In [ ]:
X_train.values[0].shape

In [ ]:
#scatter_matrix(X_test[['BB9', 'ERA_Minor']], alpha=0.2, figsize=(6, 6), diagonal='kde')

In [ ]:
X_train.values

In [ ]:
#logit = LogisticRegression()

In [ ]:
#logit.fit(X_train, y_train)

In [ ]:
X = X.drop('Unnamed: 0', axis=1)

In [ ]:
X = X.loc[:, X.dtypes!=object]

In [ ]:
y = X.pop('Rookie?')

In [ ]:
X_ver2.shape

In [ ]:
y_ver3.shape

In [ ]:
X_ver2 = X_ver2.fillna(0)

## Sklearn logit

In [ ]:
sklogit = LogisticRegression()

In [ ]:
sklogit.fit(X_resampled, y_resampled)

-----

In [ ]:
logit_mod = Logit(y_un, X_ver2).fit()

In [ ]:
logit_mod.summary()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_ver2, y_ver3)

In [ ]:
rus = RandomUnderSampler()
X_resampled, y_resampled = rus.fit_sample(X_train, y_train)
'''model_RF = RandomForestClassifier()
model_GB = GradientBoostingClassifier()
model_RF.fit(X_resampled, y_resampled)
model_GB.fit(X_resampled, y_resampled)
print model_RF.score(X_test, y_test)
print model_GB.score(X_test, y_test)
y_pred_RF = model_RF.predict(X_test)
y_pred_GB = model_GB.predict(X_test)
print precision_score(y_test, y_pred_RF)
print precision_score(y_test, y_pred_GB)
print recall_score(y_test, y_pred_RF)
print recall_score(y_test, y_pred_GB)'''